In [1]:
from web3 import Web3
import json
from tqdm import tqdm
import os
import pandas as pd

infura_url = 'https://mainnet.infura.io/v3/f0fafb6de68746c083746abf0408f3e8'
web3 = Web3(Web3.HTTPProvider(infura_url))

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
data_p1 = pd.read_pickle('data/Merged_CEX_DEX_v2_p1.pkl')
data_p2 = pd.read_pickle('data/Merged_CEX_DEX_v2_p2.pkl')
data_p3 = pd.read_pickle('data/Merged_CEX_DEX_v2_p3.pkl')

data_p1['time'] = pd.to_datetime(data_p1['time'])
data_p2['time'] = pd.to_datetime(data_p2['time'])
data_p3['time'] = pd.to_datetime(data_p3['time'])

data = pd.concat([data_p1, data_p2, data_p3])
data = data.sort_values('time')

In [7]:
block_numbers = list(data.blockNumber.unique())  # Replace with actual list

gas_price_eth = {}

if os.path.exists('gas_prices.json'):
    with open('gas_prices.json', 'r') as file:
        gas_price_eth = json.load(file)

for block_number in tqdm(block_numbers):
    if block_number not in gas_price_eth:  # Skip if already calculated
        block = web3.eth.get_block(int(block_number), full_transactions=True)
        if block and 'transactions' in block:
            total_gas_price = sum(tx['gasPrice'] * 10**-18 for tx in block['transactions'])
            average_gas_price = total_gas_price / len(block['transactions']) if block['transactions'] else 0
            gas_price_eth[block_number] = float(average_gas_price)
            
        gas_price_eth_str_keys = {str(key): value for key, value in gas_price_eth.items()}
        
        # Periodically save the dictionary to avoid losing data
        if len(gas_price_eth_str_keys) % 10 == 0:
            with open('gas_prices.json', 'w') as file:
                json.dump(gas_price_eth_str_keys, file)

# Final save to ensure no data is lost
with open('gas_prices.json', 'w') as file:
    json.dump(gas_price_eth_str_keys, file)

  0%|          | 0/120132 [00:00<?, ?it/s]

 46%|████▌     | 55292/120132 [5:23:13<4:22:14,  4.12it/s]     